# Unsupervised PPE Detection (Colab)

## 1. Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install -r requirements.txt

import sys
import os
sys.path.append('/content/drive/My Drive/ppe_detection') # Adjust this path to your project location
from ppe_detection.config import load_config
from ppe_detection.data_utils import prepare_dataset
from ppe_detection.unsupervised_trainer import UnsupervisedTrainer
from ppe_detection.discovery_processor import DiscoveryProcessor
from ppe_detection.violation_processor import ViolationProcessor

# Load the configuration
config = load_config()

## 2. Training

In [ ]:
# Training overrides
training_overrides = {
    'frozen_layers': 12, # Example of overriding a parameter
    'data_fraction': 0.5 # Example of using a subset of data
}
config['model']['frozen_layers'] = training_overrides.get('frozen_layers', config['model']['frozen_layers'])
config['training']['data_fraction'] = training_overrides.get('data_fraction', config['training']['data_fraction'])

# Prepare the dataset
data, labels = prepare_dataset(config['data_dir'], config['training']['data_fraction'])
# In a real scenario, you would create a proper data loader here
import torch
dummy_dataset = torch.utils.data.TensorDataset(torch.randn(len(data), 3, 518, 518), torch.tensor(labels))
data_loader = torch.utils.data.DataLoader(dummy_dataset, batch_size=config['training']['batch_size'])

# Initialize and run the trainer
run_training = True
if run_training:
    trainer = UnsupervisedTrainer(config)
    trainer.train(data_loader)

## 3. Discovery and Mapping

In [ ]:
# Load the fine-tuned model (optional)
model_path = 'output/checkpoints/latest_checkpoint.pt'
discovery_processor = DiscoveryProcessor(config, model_path=model_path)

# In a real scenario, you would load an image and generate masks
import numpy as np
dummy_image = np.zeros((480, 640, 3), dtype=np.uint8)
masks = discovery_processor.generate_object_masks(dummy_image)

# Manual class mapping (example)
# This step would involve visualizing the discovered clusters and assigning labels.
class_map = {1: 'person', 2: 'helmet', 3: 'vest'}
config['discovery']['class_map'] = class_map


## 4. Inference and Violation Detection

In [ ]:
violation_processor = ViolationProcessor(config)

# In a real scenario, you would process a video frame by frame
# For now, we'll use a dummy example
discovered_objects = [
    {'class_name': 'person', 'box': [100, 100, 200, 400], 'cluster_id': 1},
    {'class_name': 'vest', 'box': [120, 150, 180, 250], 'cluster_id': 3},
    {'class_name': 'person', 'box': [300, 100, 400, 400], 'cluster_id': 1},
]

labeled_objects = discovery_processor.apply_class_map(discovered_objects)
violations = violation_processor.process_violations(labeled_objects, dummy_image)
print(violations)